<a href="https://colab.research.google.com/github/ArshHp/LLM/blob/main/kg_relations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q load_dotenv langchain_community neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [4]:
from dotenv import load_dotenv
import os

# Common data processing
import json
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain



# Warning control
import warnings
warnings.filterwarnings("ignore")

In [5]:
NEO4J_URI = 'neo4j+s://ec3e319b.databases.neo4j.io:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'o1supQkFCHQPMOG1Z9Usk39RU8Wf6FzAbPhDxEVhZEo'
NEO4J_DATABASE = 'neo4j'
# Global constants
VECTOR_INDEX_NAME = 'troubleshooting_data'
VECTOR_NODE_LABEL = 'Node'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [6]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [5]:
cypher = """
  MATCH (anyChunk:Node)
  WITH anyChunk LIMIT 1
  RETURN anyChunk { .chunkSeqId, .contentType, .source} as formInfo
"""
form_info_list = kg.query(cypher)

form_info_list

[{'formInfo': {'source': 'troubleshooting_data',
   'contentType': 'Issue',
   'chunkSeqId': '101.1'}}]

In [9]:
form_info = form_info_list[0]['formInfo']

In [7]:
form_info

{'source': 'troubleshooting_data',
 'contentType': 'Issue',
 'chunkSeqId': '101.1'}

In [10]:
cypher = """
    MERGE (f:Form {formId: $formInfoParam.chunkSeqId })
      ON CREATE
        SET f.source = $formInfoParam.source
        SET f.contentType = $formInfoParam.contentType
        SET f.chunkSeqId = $formInfoParam.chunkSeqId
"""

kg.query(cypher, params={'formInfoParam': form_info})

[]

In [11]:
kg.query("MATCH (f:Form) RETURN count(f) as formCount")

[{'formCount': 2}]

In [ ]:
cypher = """
  MATCH (from_same_form:Chunk)
    WHERE from_same_form.formId = $formIdParam
  RETURN from_same_form {.contentType, .chunkId, .chunkSeqId } as chunkInfo
    LIMIT 10
"""

kg.query(cypher, params={'formIdParam': form_info['chunkSeqId']})

[]

In [ ]:
cypher = """
  MATCH (n1:Chunk {contentType: 'Issue'}), (n2:Chunk {contentType: 'Cause'})
  CREATE (n1)-[:CAUSE]->(n2)
  RETURN n1, n2;
"""
kg.query(cypher)



[{'n1': {'textEmbedding': [-0.5764299631118774,
    -0.1626749187707901,
    -0.4972211420536041,
    -0.6699290871620178,
    0.690149188041687,
    0.6723228096961975,
    0.20480534434318542,
    0.8233212828636169,
    -0.6031633615493774,
    -0.6725948452949524,
    -0.5327672958374023,
    0.39590248465538025,
    -0.36467522382736206,
    0.13564404845237732,
    -0.637062668800354,
    -0.4089118242263794,
    0.445322722196579,
    0.25133994221687317,
    1.220962643623352,
    0.10166186094284058,
    -0.5434260368347168,
    -0.38807618618011475,
    0.35648566484451294,
    -0.4675583243370056,
    -0.8002737164497375,
    0.7734646201133728,
    0.6234778165817261,
    -0.11018459498882294,
    -0.4386609196662903,
    -0.4692823588848114,
    0.38225889205932617,
    0.963336169719696,
    -0.6378964781761169,
    -0.4959437847137451,
    -0.46352142095565796,
    0.9024577736854553,
    -0.4606819152832031,
    -0.8461264967918396,
    -0.5722541809082031,
    0.545153

In [ ]:
#Remove Duplicate CAUSE relations
cypher = """ MATCH (n1)-[rel:CAUSE]->(n2)
WITH n1, n2, TYPE(rel) AS relType, COLLECT(rel) AS rels
WHERE SIZE(rels) > 1
FOREACH (r IN rels[1..] | DELETE r)
"""

kg.query(cypher)


[]

In [12]:
cypher = """
  MATCH (n1:Node {contentType: 'Issue'}), (n2:Node {contentType: 'Cause'})
WHERE substring(n1.chunkSeqId, 0, 3) = substring(n2.chunkSeqId, 0, 3)
CREATE (n1)-[:CAUSE]->(n2)
RETURN n1, n2;

"""

kg.query(cypher)

[{'n1': {'textEmbedding': [-0.5764299631118774,
    -0.1626749187707901,
    -0.4972211420536041,
    -0.6699290871620178,
    0.690149188041687,
    0.6723228096961975,
    0.20480534434318542,
    0.8233212828636169,
    -0.6031633615493774,
    -0.6725948452949524,
    -0.5327672958374023,
    0.39590248465538025,
    -0.36467522382736206,
    0.13564404845237732,
    -0.637062668800354,
    -0.4089118242263794,
    0.445322722196579,
    0.25133994221687317,
    1.220962643623352,
    0.10166186094284058,
    -0.5434260368347168,
    -0.38807618618011475,
    0.35648566484451294,
    -0.4675583243370056,
    -0.8002737164497375,
    0.7734646201133728,
    0.6234778165817261,
    -0.11018459498882294,
    -0.4386609196662903,
    -0.4692823588848114,
    0.38225889205932617,
    0.963336169719696,
    -0.6378964781761169,
    -0.4959437847137451,
    -0.46352142095565796,
    0.9024577736854553,
    -0.4606819152832031,
    -0.8461264967918396,
    -0.5722541809082031,
    0.545153

In [13]:
cypher = """
  MATCH (n1:Node {contentType: 'Cause'}), (n2:Node {contentType: 'Solution'})
WHERE substring(n1.chunkSeqId, 0, 3) = substring(n2.chunkSeqId, 0, 3)
CREATE (n1)-[:SOLUTION]->(n2)
RETURN n1, n2;

"""

kg.query(cypher)

[{'n1': {'textEmbedding': [-1.173985481262207,
    -0.5146881341934204,
    -0.35214540362358093,
    -0.7574208974838257,
    1.1248955726623535,
    0.6059234738349915,
    0.6697630286216736,
    1.1750086545944214,
    -1.1400864124298096,
    -0.6592382192611694,
    -0.39502382278442383,
    0.61416095495224,
    -0.17701351642608643,
    0.2563457489013672,
    -0.6057802438735962,
    -0.30164554715156555,
    0.09048135578632355,
    0.05817967653274536,
    1.0249265432357788,
    0.3406796455383301,
    -0.8749440312385559,
    -0.4782107472419739,
    0.983141303062439,
    -0.24753305315971375,
    -0.42464444041252136,
    0.6227192282676697,
    0.04504994675517082,
    0.21371492743492126,
    -0.07817570120096207,
    -0.416908860206604,
    0.27013474702835083,
    0.39752453565597534,
    -0.4620133936405182,
    -0.5117992758750916,
    -0.38117945194244385,
    0.7351306676864624,
    -0.21126966178417206,
    -0.2793056070804596,
    -0.3326035141944885,
    0.668

In [14]:
cypher = """
  MATCH (n1:Node {contentType: 'Issue'}), (n2:Node {contentType: 'Solution'})
WHERE substring(n1.chunkSeqId, 0, 3) = substring(n2.chunkSeqId, 0, 3)
CREATE (n1)-[:SOLUTION]->(n2)
RETURN n1, n2;

"""

kg.query(cypher)

[{'n1': {'textEmbedding': [-0.5764299631118774,
    -0.1626749187707901,
    -0.4972211420536041,
    -0.6699290871620178,
    0.690149188041687,
    0.6723228096961975,
    0.20480534434318542,
    0.8233212828636169,
    -0.6031633615493774,
    -0.6725948452949524,
    -0.5327672958374023,
    0.39590248465538025,
    -0.36467522382736206,
    0.13564404845237732,
    -0.637062668800354,
    -0.4089118242263794,
    0.445322722196579,
    0.25133994221687317,
    1.220962643623352,
    0.10166186094284058,
    -0.5434260368347168,
    -0.38807618618011475,
    0.35648566484451294,
    -0.4675583243370056,
    -0.8002737164497375,
    0.7734646201133728,
    0.6234778165817261,
    -0.11018459498882294,
    -0.4386609196662903,
    -0.4692823588848114,
    0.38225889205932617,
    0.963336169719696,
    -0.6378964781761169,
    -0.4959437847137451,
    -0.46352142095565796,
    0.9024577736854553,
    -0.4606819152832031,
    -0.8461264967918396,
    -0.5722541809082031,
    0.545153

In [8]:
!pip install -q apoc.math.cosineSimilarity

ERROR: Could not find a version that satisfies the requirement apoc.math.cosineSimilarity (from versions: none)
ERROR: No matching distribution found for apoc.math.cosineSimilarity


In [9]:
cypher = """
// Define your similarity threshold (e.g., 0.8 for 80%)
WITH 0.8 AS threshold

// Match two nodes with embeddings
MATCH (n1:Node), (n2:Node)
WHERE n1 <> n2  // Ensure they are not the same node

// Calculate cosine similarity using APOC procedure
WITH n1, n2, apoc.algo.cosineSimilarity(n1.textEmbedding, n2.textEmbedding) AS similarity

// Check if the similarity meets the threshold and relationship between Issue to Issue only
WHERE similarity >= threshold AND n1.contentType = 'Issue' AND n2.contentType = 'Issue'

// Create a relationship between the nodes if they meet the similarity threshold
CREATE (n1)-[:SIMILAR {similarity: similarity}]->(n2)

// Return the nodes and the similarity value
RETURN n1, n2, similarity;

"""

kg.query(cypher)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Unknown function 'apoc.algo.cosineSimilarity' (line 10, column 14 (offset: 261))
"WITH n1, n2, apoc.algo.cosineSimilarity(n1.textEmbedding, n2.textEmbedding) AS similarity"
              ^}